In [1]:
from F_Functions import *
from C_Classes import SUBMISSION_510K, SUBMISSION_DE_NOVO, SUBMISSION_PMA

/Users/adam/Projects/23.10.26_FDA_Scraper/.env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Number = 'K232699' #Example of a 510(k) with a machine-readible PDF
# Number = 'K991766' #Example of a 510(k) with a photo-copied and not machine-readible PDF

# # 510k class
# sub1 = SUBMISSION_510K(Number)
# sub1.get_510k_summary()
# sub1.get_510k_pdf()
# sub1.get_510k_IFU()
# # sub1.find_potential_predicates(sub1.PDF_Location)
# sub1.summarize()


# Number = 'DEN130045' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN130045
# Number = 'DEN180044' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN180044
# Number = 'DEN200030' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN200030

# #De Novo class
# sub2 = SUBMISSION_DE_NOVO(Number)
# # sub2.Submission_Type
# # sub2.Submission_Number
# sub2.get_De_Novo_summary()
# sub2.get_Reclassification_pdf()
# sub2.get_FDA_Review_pdf()
# # sub2.get_De_Novo_IFU()
# sub2.find_potential_predicates(sub2.PDF_Review_Location)
# sub2.summarize()


# Number = 'P220014'
# Number = 'P220004'
# Number = 'P230002'
# Number = 'P220005'

# sub = SUBMISSION_PMA(Number)
# sub.get_PMA_summary()
# sub.get_FDA_Approval_Order_pdf()
# sub.get_FDA_Summary_pdf()
# sub.get_FDA_Labeling_pdf()
# sub.summarize()


In [5]:
# https://www.fda.gov/about-fda/cdrh-transparency/evaluation-automatic-class-iii-designation-de-novo-summaries
# https://www.fda.gov/medical-devices/510k-clearances/downloadable-510k-files


In [2]:
# Download FDA Cleared 510(k) Submissions
FDA_files = ['pmnlstmn','pmn96cur','pmn9195','pmn8690','pmn8185','pmn7680']
FDA_local_files = [download_FDA_file(file) for file in FDA_files]
FDA_dataframes = [create_FDA_dataframe(file) for file in FDA_local_files]
df = pd.concat(FDA_dataframes)
df.shape


# Sandox

In [1]:
import re
import sys
import requests
import numpy as np
import pandas as pd
import requests, zipfile, io
from PyPDF2 import PdfReader 
from bs4 import BeautifulSoup

from F_Functions import *
from C_Classes import *

/Users/adam/Projects/23.10.26_FDA_Scraper/.env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
def initialize_class_for_submission(submission_number):
    if  re.match('[K,k]\d{6}', submission_number): 
        sub = SUBMISSION_510K(submission_number)
        sub.get_510k_summary()
        sub.get_510k_pdf()
        # sub.get_510k_IFU()
        sub.find_potential_predicates(sub.PDF_Location)
    elif re.match('[D,d][E,e][N,n]\d{6}', submission_number): 
        # #De Novo class
        sub = SUBMISSION_DE_NOVO(submission_number)
        sub.get_De_Novo_summary()
        sub.get_Reclassification_pdf()
        sub.get_FDA_Review_pdf()
        # sub.get_De_Novo_IFU()
        sub.find_potential_predicates(sub.PDF_Review_Location)
        sub.summarize()
    else:
        print("This does not look like a 510k Submission number!")
        sub = None

    return sub

In [5]:
starting_submission = 'K232699'
starting_sub = initialize_class_for_submission(starting_submission)

In [6]:
first_layer = []
list_of_edges = []
for predicate in starting_sub.Potential_Predicates:
    print(predicate)
    predicate_sub = initialize_class_for_submission(predicate)
    list_of_edges.append(starting_sub.Submission_Number + ' -> ' + predicate_sub.Submission_Number)
    first_layer.append(predicate_sub)

K210392
K201446
DEN230003


AttributeError: 'SUBMISSION_DE_NOVO' object has no attribute 'FDA_Review_URL'

In [ ]:
list_of_edges